In [229]:
import pandas as pd
import numpy as np
import requests
import ast 
from datetime import datetime, timedelta

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [230]:
class Extraccion: 
    # primero tenemos que crear el constructor con las variables globales que usaremos. Están son variables que podrán cambiar cada vez que llamemos a la clase. 
    def __init__(self, paises):

        # definimos cada una de las variables. Recordamos que tenemos que usar el método .self para definirlas y que la clase entienda que las podremos usar en otros métodos de la clase. 

        self.paises = paises

    # definimos nuestro primer método. El que nos hará la llamada a la API
    # Si comparamos, veremos que la función es exactamente igual a la que hicimos en la clase anterior, solo que con unos pequeños cambios. En este caso incluimos el parámetro self
    def cargar_ataques(self, ruta_archivo):

        self.ruta_archivo = ruta_archivo

        df_ataques = pd.read_csv(ruta_archivo, index_col=0)

        lista_paises = []
        
        for k in self.paises.keys():

            lista_paises.append(k.lower())

        df_ataques = df_ataques[df_ataques['country'].isin(lista_paises)]

        return df_ataques


    
    def llamada_API(self, producto):

        self.producto = producto

        df_final = pd.DataFrame()

        for key, value in self.paises.items():
        
            url = f'http://www.7timer.info/bin/api.pl?lon=-{value[1]}&lat={value[0]}&product={producto}&output=json'

            response = requests.get(url=url)
            codigo_estado = response.status_code
            razon_estado = response.reason
            if codigo_estado == 200:
                print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
            elif codigo_estado == 402:
                print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            elif codigo_estado == 404:
                print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            else:
                print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            
            df = pd.json_normalize(response.json()["dataseries"])
            df['latitud'] = value[0]
            df['longitud'] = value[1]
            df['pais'] = key
            df_final = pd.concat([df_final,df], axis=0, ignore_index=True) 
         
        return df_final
    
    # definimos un nuevo método, el de limpiar los datos obtenidos de la llamada a la API para el producto civil. 
    def limpiar_columnas_meteo(self, df_meteo):

        x = df_meteo['wind_profile'].apply(pd.Series)

        for i in range(len(x.columns)): 

            # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = 'windspeed_' + str(x[i].apply(pd.Series)['layer'][0])

            # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(x[i].apply(pd.Series)["speed"] )

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_meteo.insert(i, nombre, valores)

        x = df_meteo['rh_profile'].apply(pd.Series)

        for i in range(len(x.columns)): 

        # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = 'rh_' + str(x[i].apply(pd.Series)['layer'][0] )
            # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(x[i].apply(pd.Series)["rh"] )

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_meteo.insert(i, nombre, valores)

        return df_meteo

    def meteo_medias_fecha(self, df_meteo):

        df_meteo_medias = df_meteo.groupby('pais').apply(lambda x: x.mean())

        df_meteo_medias.reset_index(inplace=True)

        df_meteo_medias['pais'] = df_meteo_medias['pais'].apply(lambda x: x.lower())

        hoy = datetime.now()
        
        hoy = datetime.strftime(hoy, '%Y-%m-%d')
        
        df_meteo_medias["fecha_descarga"] = hoy
        
        return df_meteo_medias 


    def juntar_dfs(self, df_meteo_medias, df_ataques): 

        df_ataques_meteo =  pd.merge(left = df_ataques, right= df_meteo_medias, how= "left", left_on= "country", right_on= "pais")

        # guardamos los dato
        df_ataques_meteo.to_pickle('datos/ataques_meteo.pkl')
        df_ataques_meteo.to_csv('datos/ataques_meteo.csv')

        return df_ataques_meteo
    

    def chequear_datos(self, df_ataques_meteo): 
    
        print("Las columnas son:", "\n")
        print(list(df_ataques_meteo.columns))
        print("-----------------------------------------")

        print("Los tipos de datos que tenemos son:", "\n")
        print(df_ataques_meteo.dtypes)
        print("-----------------------------------------")

        print("El porcentaje de nulos:", "\n")
        print((df_ataques_meteo.isnull().sum() / df_ataques_meteo.shape[0]) *  100)


Comprobamos que nuestra Clase funciona correctamente

In [231]:
paises = {"USA" : [39.7837304,-100.445882], "AUSTRALIA" : [-24.7761086, 134.755], "SOUTH AFRICA" : 
    [-28.8166236, 24.991639], "NEW ZEALAND" : [-41.5000831, 172.8344077], "PAPUA NEW GUINEA" : [-5.6816069, 144.2489081]}
producto = "meteo"
ruta_ataques = '../../datos/attacks_final.csv'

In [232]:
api = Extraccion(paises)

In [233]:
api

In [234]:
df_meteo = api.llamada_API(producto)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


In [235]:
df_meteo.head(1)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,pais
0,3,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 8}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 95, 'speed': ...",24,2,5,1012,none,0,0,85,3,39.78373,-100.445882,USA


In [236]:
df_meteo = api.limpiar_columnas_meteo(df_meteo)

In [237]:
df_meteo

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,pais
0,8,11,14,13,10,5,2,2,2,1,...,5,1012,none,0,0,85,3,39.783730,-100.445882,USA
1,9,11,13,14,11,9,3,3,2,2,...,6,1013,none,0,0,65,3,39.783730,-100.445882,USA
2,9,11,14,14,12,10,6,3,2,2,...,6,1014,rain,0,0,25,2,39.783730,-100.445882,USA
3,11,15,14,13,13,11,5,1,1,1,...,7,1014,rain,1,0,35,2,39.783730,-100.445882,USA
4,11,15,15,13,11,10,5,1,1,0,...,7,1014,rain,1,0,25,2,39.783730,-100.445882,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,13,12,2,-1,3,1,-3,-4,-4,-4,...,11,1009,none,3,0,105,4,-5.681607,144.248908,PAPUA NEW GUINEA
316,12,10,1,-1,3,1,-3,-4,-4,-4,...,11,1010,none,3,0,105,4,-5.681607,144.248908,PAPUA NEW GUINEA
317,13,11,3,-1,3,0,-3,-4,-4,-4,...,11,1012,none,3,0,105,4,-5.681607,144.248908,PAPUA NEW GUINEA
318,14,13,1,0,3,0,-3,-4,-4,-4,...,11,1012,rain,3,0,105,4,-5.681607,144.248908,PAPUA NEW GUINEA


In [238]:
df_meteo_medias = api.meteo_medias_fecha(df_meteo)

/tmp/ipykernel_7987/215706168.py:91: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_meteo_medias = df_meteo.groupby('pais').apply(lambda x: x.mean())


In [239]:
df_meteo_medias

,pais,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,...,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,fecha_descarga
0,australia,13.000000,12.453125,8.156250,4.328125,1.859375,0.578125,0.109375,-0.546875,-0.718750,...,2.671875,10.250000,1023.187500,1.859375,0.0,1.876955e+152,3.203125,-24.776109,134.755000,2022-09-17
1,new zealand,11.859375,12.984375,7.984375,4.593750,3.421875,2.125000,3.359375,4.640625,5.703125,...,7.921875,11.218750,1025.625000,2.656250,0.0,3.518988e+167,3.640625,-41.500083,172.834408,2022-09-17
2,papua new guinea,13.281250,11.250000,6.812500,2.234375,0.796875,-0.703125,-2.890625,-3.703125,-3.937500,...,2.015625,10.875000,1011.187500,1.968750,0.0,1.954924e+163,3.546875,-5.681607,144.248908,2022-09-17
3,south africa,12.765625,11.875000,11.234375,11.265625,10.515625,9.546875,8.781250,7.578125,6.625000,...,2.609375,12.593750,1017.671875,4.890625,0.0,5.317974e+145,3.187500,-28.816624,24.991639,2022-09-17
4,usa,11.218750,12.890625,11.859375,10.093750,9.250000,9.000000,7.000000,4.593750,3.796875,...,-0.250000,8.078125,1015.437500,3.234375,0.0,1.338321e+163,2.406250,39.783730,-100.445882,2022-09-17


In [240]:
df_ataques = api.cargar_ataques(ruta_ataques)

In [241]:
df_ataques.head(1)

,year,mes,country,type,age,sex,species,fatal
3,1976,Unknown,australia,Unprovoked,27.28,F,UNKNOWN,y


In [242]:
df_ataques_meteo = api.juntar_dfs(df_meteo_medias, df_ataques)

In [243]:
df_ataques_meteo.sample(10)

,year,mes,country,type,age,sex,species,fatal,pais,rh_950mb,...,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,fecha_descarga
4046,1906,Unknown,australia,Invalid,18.00,M,UNKNOWN,unknown,australia,13.000000,...,2.671875,10.250000,1023.187500,1.859375,0.0,1.876955e+152,3.203125,-24.776109,134.755000,2022-09-17
1751,1998,Jan,australia,Unprovoked,27.28,M,UNKNOWN,n,australia,13.000000,...,2.671875,10.250000,1023.187500,1.859375,0.0,1.876955e+152,3.203125,-24.776109,134.755000,2022-09-17
3116,1959,Sep,usa,Invalid,27.28,M,UNKNOWN,unknown,usa,11.218750,...,-0.250000,8.078125,1015.437500,3.234375,0.0,1.338321e+163,2.406250,39.783730,-100.445882,2022-09-17
3810,1927,Feb,australia,Unprovoked,27.28,M,UNKNOWN,n,australia,13.000000,...,2.671875,10.250000,1023.187500,1.859375,0.0,1.876955e+152,3.203125,-24.776109,134.755000,2022-09-17
2862,1964,Jan,south africa,Unprovoked,27.28,M,UNKNOWN,n,south africa,12.765625,...,2.609375,12.593750,1017.671875,4.890625,0.0,5.317974e+145,3.187500,-28.816624,24.991639,2022-09-17
3077,1960,Feb,australia,Unprovoked,32.00,M,UNKNOWN,n,australia,13.000000,...,2.671875,10.250000,1023.187500,1.859375,0.0,1.876955e+152,3.203125,-24.776109,134.755000,2022-09-17
2554,1975,Mar,usa,Unprovoked,14.00,M,UNKNOWN,n,usa,11.218750,...,-0.250000,8.078125,1015.437500,3.234375,0.0,1.338321e+163,2.406250,39.783730,-100.445882,2022-09-17
140,2017,Jul,usa,Unprovoked,46.00,M,BULL SHARK,n,usa,11.218750,...,-0.250000,8.078125,1015.437500,3.234375,0.0,1.338321e+163,2.406250,39.783730,-100.445882,2022-09-17
231,2016,Jul,australia,Provoked,31.00,M,UNKNOWN,n,australia,13.000000,...,2.671875,10.250000,1023.187500,1.859375,0.0,1.876955e+152,3.203125,-24.776109,134.755000,2022-09-17
991,2008,May,south africa,Provoked,24.00,M,UNKNOWN,n,south africa,12.765625,...,2.609375,12.593750,1017.671875,4.890625,0.0,5.317974e+145,3.187500,-28.816624,24.991639,2022-09-17


In [244]:
api.chequear_datos(df_ataques_meteo)

Las columnas son: 

['year', 'mes', 'country', 'type', 'age', 'sex', 'species', 'fatal', 'pais', 'rh_950mb', 'rh_900mb', 'rh_850mb', 'rh_800mb', 'rh_750mb', 'rh_700mb', 'rh_650mb', 'rh_600mb', 'rh_550mb', 'rh_500mb', 'rh_450mb', 'rh_400mb', 'rh_350mb', 'rh_300mb', 'rh_250mb', 'rh_200mb', 'windspeed_950mb', 'windspeed_900mb', 'windspeed_850mb', 'windspeed_800mb', 'windspeed_750mb', 'windspeed_700mb', 'windspeed_650mb', 'windspeed_600mb', 'windspeed_550mb', 'windspeed_500mb', 'windspeed_450mb', 'windspeed_400mb', 'windspeed_350mb', 'windspeed_300mb', 'windspeed_250mb', 'windspeed_200mb', 'timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud', 'temp2m', 'lifted_index', 'rh2m', 'msl_pressure', 'prec_amount', 'snow_depth', 'wind10m.direction', 'wind10m.speed', 'latitud', 'longitud', 'fecha_descarga']
-----------------------------------------
Los tipos de datos que tenemos son: 

year                   int64
mes                   object
country               object
type              